# XSOLIS Finance/Utilization Demo

Synthetic finance & utilization analysis with KPIs and visuals.

In [ ]:

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

df = pd.read_csv("data/cases.csv", parse_dates=["admit_date","discharge_date"])
df.head()


## Basic KPIs

In [ ]:

total_cases = len(df)
avg_los = df["length_of_stay"].mean()
readmit_rate = df["readmission_30d"].mean()
mismatch_rate = df["mismatch"].mean()

provider_total = df["provider_cost"].sum()
payer_total = df["payer_cost"].sum()
budget_total = df["budget_cost"].sum()

variance_budget_total = provider_total - budget_total
variance_payer_provider_total = provider_total - payer_total

kpis = {
    "Total Cases": total_cases,
    "Avg LOS (days)": round(avg_los, 2),
    "Readmission Rate": f"{readmit_rate:.1%}",
    "Mismatch Rate": f"{mismatch_rate:.1%}",
    "Total Provider Cost ($)": round(provider_total, 2),
    "Total Payer Cost ($)": round(payer_total, 2),
    "Total Budget Cost ($)": round(budget_total, 2),
    "Provider vs Budget Variance ($)": round(variance_budget_total, 2),
    "Provider vs Payer Variance ($)": round(variance_payer_provider_total, 2),
}
kpis


## LOS Trend by Month

In [ ]:

df["admit_month"] = df["admit_date"].dt.to_period("M").dt.to_timestamp()
los_by_month = df.groupby("admit_month")["length_of_stay"].mean()

plt.figure()
los_by_month.plot(kind="line", title="Average Length of Stay by Month")
plt.xlabel("Month"); plt.ylabel("Average LOS (days)"); plt.tight_layout(); plt.show()


## Provider vs Payer Cost Distribution

In [ ]:

plt.figure()
df[["provider_cost","payer_cost"]].plot(kind="box", title="Cost Distribution: Provider vs Payer")
plt.ylabel("USD"); plt.tight_layout(); plt.show()


## Mismatch Rate by Provider

In [ ]:

mismatch_rate = df.groupby("provider")["mismatch"].mean().sort_values()

plt.figure()
mismatch_rate.plot(kind="barh", title="Payer/Provider Cost Mismatch Rate by Provider")
plt.xlabel("Mismatch Rate"); plt.tight_layout(); plt.show()


## Avg Provider Variance vs Budget by DRG

In [ ]:

variance_by_drg = (df.assign(variance=df["provider_cost"] - df["budget_cost"])
                     .groupby("drg")["variance"].mean().sort_values(ascending=False))

plt.figure()
variance_by_drg.plot(kind="bar", title="Avg Provider Variance vs Budget by DRG")
plt.ylabel("USD"); plt.tight_layout(); plt.show()
